## Get Best-Performing Hyper Parameters

In [174]:
# MONGO_COLLECTION = "CB_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "CB_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

MONGO_COLLECTION = "CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD"

# MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "SC_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

MONGO_COLLECTION

'CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD'

In [175]:
assert MONGO_COLLECTION.endswith("VD")

In [176]:
cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type"]
if "INSENS" in MONGO_COLLECTION:
    cols = [c for c in cols if c != "loss_type"]

elif "PCPTRN" in MONGO_COLLECTION:
    cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "learning_rate", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight"]
    #"min_feat_freq"]
# cols

In [181]:
import numpy as np

np.unique([5,3,2,5,10])

array([ 2,  3,  5, 10])

In [177]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [178]:
df = pd.DataFrame(results)
# df.tail(3)[["asof"] + cols]

In [179]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,learning_rate,best_max_parses,max_update_items,initial_weight
26,0.741512,0.786416,0.701460,2019-06-23 22:55:15.285000,1,0.005,300,2,0.01
4,0.741243,0.789040,0.698905,2019-06-23 21:59:15.302000,2,0.050,300,2,0.01
27,0.740812,0.788066,0.698905,2019-06-23 22:56:22.165000,1,0.010,300,2,0.01
31,0.740081,0.787804,0.697810,2019-06-23 23:01:01.125000,1,1.000,300,2,0.01
29,0.740066,0.789169,0.696715,2019-06-23 22:58:45.089000,1,0.100,300,2,0.01


In [180]:
MONGO_COLLECTION

'CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD'

In [171]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
37,0.746014,0.777294,0.717153,2019-06-18 01:55:37.752000,2,0.0025,300,1,0.01,ml,1
57,0.745665,0.789796,0.706204,2019-06-18 02:38:17.475000,1,0.0100,300,1,0.01,ml,1
35,0.745506,0.774067,0.718978,2019-06-18 01:51:39.666000,2,0.0005,300,1,0.01,ml,2
53,0.745016,0.776415,0.716058,2019-06-18 02:30:03.563000,1,0.0025,300,1,0.01,ml,1
33,0.744965,0.769141,0.722263,2019-06-18 01:47:36.756000,2,0.0005,300,1,0.01,ml,1


In [172]:
df.max_update_items.unique()

array([1])

In [173]:
# Best ones that are not 2
# df[df.best_top_n != 2][cols].head()